In [17]:
import os
import numpy as np
import pandas as pd
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
from sklearn.model_selection import train_test_split
import pickle
class PositionalEncoding(nn.Module):
    def __init__(self, d_model, max_len=1000):
        super().__init__()
        pe = torch.zeros(max_len, d_model)
        pos = torch.arange(0, max_len).unsqueeze(1).float()
        div = torch.exp(torch.arange(0, d_model, 2).float() * -(np.log(10000.0) / d_model))
        pe[:, 0::2] = torch.sin(pos * div)
        pe[:, 1::2] = torch.cos(pos * div)
        self.register_buffer('pe', pe.unsqueeze(0))

    def forward(self, x):
        # x: (B, L, D)
        return x + self.pe[:, :x.size(1)]

class TransformerVAE(nn.Module):
    def __init__(self, input_dim, seq_len, d_model=128, nhead=4, num_layers=2, latent_dim=64):
        super().__init__()
        self.seq_len = seq_len
        self.input_linear = nn.Linear(input_dim, d_model)
        self.pos_enc = PositionalEncoding(d_model, max_len=seq_len)
        
        encoder_layer = nn.TransformerEncoderLayer(d_model, nhead, batch_first=True) # Use batch_first
        self.encoder = nn.TransformerEncoder(encoder_layer, num_layers)
        
        # Output of encoder will be (batch_size, seq_len, d_model)
        self.fc_mu = nn.Linear(d_model * seq_len, latent_dim)
        self.fc_logvar = nn.Linear(d_model * seq_len, latent_dim)
        
        self.fc_latent_to_d_model = nn.Linear(latent_dim, d_model) # Project latent_dim to d_model for decoder initial input
        
        decoder_layer = nn.TransformerDecoderLayer(d_model, nhead, batch_first=True) # Use batch_first
        self.decoder = nn.TransformerDecoder(decoder_layer, num_layers)
        self.output_linear = nn.Linear(d_model, input_dim)

    def encode(self, x):
        # x: (B, L, D_input)
        h = self.input_linear(x) # (B, L, d_model)
        h_with_pos = self.pos_enc(h) # (B, L, d_model)
        
        # TransformerEncoder expects (batch_size, seq_len, features) with batch_first=True
        encoder_output = self.encoder(h_with_pos) # (B, L, d_model)
        
        # Flatten the encoder output for mu and logvar layers
        flat_encoder_output = encoder_output.contiguous().view(encoder_output.size(0), -1)
        
        mu = self.fc_mu(flat_encoder_output)
        logvar = self.fc_logvar(flat_encoder_output)
        
        return mu, logvar, encoder_output # Return encoder_output as memory

    def reparameterize(self, mu, logvar):
        std = torch.exp(0.5 * logvar)
        eps = torch.randn_like(std)
        return mu + eps * std

    def decode(self, z, encoder_memory):
        # z: (B, latent_dim)
        # encoder_memory: (B, L, d_model) from encoder output

        # Create the initial target sequence for the decoder.
        # This is crucial for sequence generation.
        # A common approach for VAEs is to project the latent 'z'
        # and repeat it for all time steps to form the target sequence,
        # or use a special start token.
        
        # Here, we project z to d_model, then expand it to seq_len for simplicity.
        # This assumes 'z' contains all information for the sequence generation.
        # A more complex setup might involve an explicit <SOS> token.
        
        # Project latent 'z' to d_model for each time step
        # (B, latent_dim) -> (B, d_model) -> expand to (B, seq_len, d_model)
        initial_tgt_input = self.fc_latent_to_d_model(z).unsqueeze(1).repeat(1, self.seq_len, 1)
        
        # Add positional encoding to the initial target sequence
        tgt_with_pos = self.pos_enc(initial_tgt_input) # (B, L, d_model)
        
        # Decoder takes (tgt, memory). Both should be (B, L, D) with batch_first=True
        # For standard Transformer Decoder, tgt is the sequence being built, and memory is from encoder.
        # Here, we use a fixed initial_tgt_input as the query for the decoder.
        out = self.decoder(tgt_with_pos, encoder_memory) # (B, L, d_model)
        
        return self.output_linear(out) # (B, L, input_dim)


class KeypointDataset(Dataset):
    def __init__(self, data_list, max_seq_len, input_dim, scaler=None):
        """
        Args:
            data_list (list): 包含所有 STS 序列 numpy 数组的列表。
                              每个数组应已处理为 (original_len, input_dim)。
            max_seq_len (int): 序列的最大长度。
            input_dim (int): 每个时间步的特征维度 (e.g., 50 for 25 keypoints).
            scaler (sklearn.preprocessing.Scaler): 用于归一化的 scaler 对象，
                                                 训练时传入 None，测试时传入训练好的 scaler。
        """
        self.data_list = data_list
        self.max_seq_len = max_seq_len
        self.input_dim = input_dim
        self.scaler = scaler

        if self.scaler is None:
            print("Fitting scaler on entire dataset for normalization...")
            all_data = np.vstack([seq for seq in data_list])
            self.scaler = MinMaxScaler()
            self.scaler.fit(all_data)
            print("Scaler fitted.")
        
        self.processed_data = []
        for seq_np in self.data_list:
            normalized_seq = self.scaler.transform(seq_np)

            if normalized_seq.shape[0] < self.max_seq_len:
                pad = np.zeros((self.max_seq_len - normalized_seq.shape[0], self.input_dim), dtype=np.float32)
                processed_seq = np.vstack([normalized_seq, pad])
            else:
                processed_seq = normalized_seq[:self.max_seq_len]
            self.processed_data.append(torch.tensor(processed_seq, dtype=torch.float32))

    def __len__(self):
        return len(self.processed_data)

    def __getitem__(self, idx):
        return self.processed_data[idx]

    def get_scaler(self):
        return self.scaler

SCALER_SAVE_PATH = 'scaler_segmented.pkl'

NUM_KPT = 25

D_MODEL = 128
NHEAD = 4
NUM_LAYERS = 2
LATENT_DIM = 64

LEARNING_RATE = 1e-4
BATCH_SIZE = 32

DEVICE = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {DEVICE}")

    MSE = F.mse_loss(recon_x, x, reduction='sum') # reduction='sum' for KL divergence
    
    # D_KL = -0.5 * sum(1 + log_sigma^2 - mu^2 - sigma^2)
    KL_D = -0.5 * torch.sum(1 + logvar - mu.pow(2) - logvar.exp())
    
    return MSE + beta * KL_D, MSE, KL_D

import torch.nn.functional as F

def train_vae_model():
    print("加载并预处理所有 STS 关键点数据...")
    all_sequences = []
    for filename in os.listdir(DATA_DIR):
        if filename.endswith('.csv'):
            filepath = os.path.join(DATA_DIR, filename)
            try:
                df = pd.read_csv(filepath, header=2, engine='python', on_bad_lines='skip')
                keypoint_data = df.iloc[:, 2:2+INPUT_DIM].to_numpy(dtype=np.float32)
                    all_sequences.append(keypoint_data)
                else:
                    print(f"Skipping empty or malformed file: {filename}")
            except Exception as e:
                print(f"Error reading {filename}: {e}")
                continue

    if not all_sequences:
        print("未找到任何有效的 STS 关键点数据。请检查 DATA_DIR 和 CSV 文件格式。")
        return

    train_sequences, val_sequences = train_test_split(
        all_sequences, test_size=0.2, random_state=42
    )
    print(f"训练序列数量: {len(train_sequences)}, 验证序列数量: {len(val_sequences)}")

    train_dataset = KeypointDataset(train_sequences, SEQ_LEN, INPUT_DIM)

    train_loader = DataLoader(train_dataset, batch_size=BATCH_SIZE, shuffle=True)
    val_loader = DataLoader(val_dataset, batch_size=BATCH_SIZE, shuffle=False)

    print("初始化模型...")
    model = TransformerVAE(INPUT_DIM, SEQ_LEN, D_MODEL, NHEAD, NUM_LAYERS, LATENT_DIM).to(DEVICE)
    optimizer = optim.Adam(model.parameters(), lr=LEARNING_RATE)

    print("开始训练模型...")
    best_val_loss = float('inf')

    for epoch in range(EPOCHS):
        model.train()
        train_loss = 0
        train_recon_loss = 0
        train_kl_loss = 0

        for batch_idx, data in enumerate(train_loader):
            data = data.to(DEVICE) # (B, L, D)

            optimizer.zero_grad()
            
            mu, logvar, encoder_output = model.encode(data)
            z = model.reparameterize(mu, logvar)
            
            loss, recon_loss_val, kl_loss_val = vae_loss(recon_batch, data, mu, logvar)
            
            loss.backward()
            optimizer.step()

            train_loss += loss.item()
            train_recon_loss += recon_loss_val.item()
            train_kl_loss += kl_loss_val.item()

            if (batch_idx + 1) % LOG_INTERVAL == 0:
                print(f"Epoch {epoch+1}/{EPOCHS}, Batch {batch_idx+1}/{len(train_loader)} | "
                      f"Train Loss: {train_loss / (batch_idx+1):.4f} | "
                      f"Recon Loss: {train_recon_loss / (batch_idx+1):.4f} | "
                      f"KL Loss: {train_kl_loss / (batch_idx+1):.4f}")

        avg_train_recon_loss = train_recon_loss / len(train_loader.dataset)
        avg_train_kl_loss = train_kl_loss / len(train_loader.dataset)
        print(f"Epoch {epoch+1} Complete | Avg Train Loss: {avg_train_loss:.4f} | "
              f"Avg Train Recon Loss: {avg_train_recon_loss:.4f} | "
              f"Avg Train KL Loss: {avg_train_kl_loss:.4f}")

        model.eval()
        val_loss = 0
        val_recon_loss = 0
        val_kl_loss = 0
        with torch.no_grad():
            for data in val_loader:
                data = data.to(DEVICE)
                mu, logvar, encoder_output = model.encode(data)
                z = model.reparameterize(mu, logvar)
                
                loss, recon_loss_val, kl_loss_val = vae_loss(recon_batch, data, mu, logvar)
                
                val_loss += loss.item()
                val_recon_loss += recon_loss_val.item()
                val_kl_loss += kl_loss_val.item()
        
        avg_val_loss = val_loss / len(val_loader.dataset)
        avg_val_recon_loss = val_recon_loss / len(val_loader.dataset)
        avg_val_kl_loss = val_kl_loss / len(val_loader.dataset)
        print(f"Epoch {epoch+1} Validation | Avg Val Loss: {avg_val_loss:.4f} | "
              f"Avg Val Recon Loss: {avg_val_recon_loss:.4f} | "
              f"Avg Val KL Loss: {avg_val_kl_loss:.4f}")

        if avg_val_loss < best_val_loss:
            best_val_loss = avg_val_loss
            torch.save(model.state_dict(), MODEL_SAVE_PATH)
            print(f"✔ 最佳模型已保存，验证损失: {best_val_loss:.4f}")


        with open(SCALER_SAVE_PATH, 'wb') as f:
        print(f"Scaler 已保存到 {SCALER_SAVE_PATH}")

    print("训练完成！")

if __name__ == "__main__":
    train_vae_model()

Using device: cuda
加载并预处理所有 STS 关键点数据...
训练序列数量: 323, 验证序列数量: 81
Fitting scaler on entire dataset for normalization...
Scaler fitted.
初始化模型...
开始训练模型...
Epoch 1 Complete | Avg Train Loss: 2385.8835 | Avg Train Recon Loss: 2383.4447 | Avg Train KL Loss: 2438.8123
Epoch 1 Validation | Avg Val Loss: 741.7767 | Avg Val Recon Loss: 736.3333 | Avg Val KL Loss: 5443.3860
✔ 最佳模型已保存，验证损失: 741.7767
Scaler 已保存到 scaler_segmented.pkl
Epoch 2 Complete | Avg Train Loss: 873.7699 | Avg Train Recon Loss: 867.4068 | Avg Train KL Loss: 6363.0289
Epoch 2 Validation | Avg Val Loss: 614.1824 | Avg Val Recon Loss: 606.2773 | Avg Val KL Loss: 7905.0868
✔ 最佳模型已保存，验证损失: 614.1824
Scaler 已保存到 scaler_segmented.pkl
Epoch 3 Complete | Avg Train Loss: 777.8743 | Avg Train Recon Loss: 769.9543 | Avg Train KL Loss: 7920.0485
Epoch 3 Validation | Avg Val Loss: 555.7573 | Avg Val Recon Loss: 547.1304 | Avg Val KL Loss: 8626.9402
✔ 最佳模型已保存，验证损失: 555.7573
Scaler 已保存到 scaler_segmented.pkl
Epoch 4 Complete | Avg Train Loss: 

In [18]:
try:
    temp_model = TransformerVAE(INPUT_DIM, SEQ_LEN, D_MODEL, NHEAD, NUM_LAYERS, LATENT_DIM).to(DEVICE)
    temp_model.load_state_dict(torch.load(MODEL_SAVE_PATH, map_location=DEVICE, weights_only=True))
    print("✔ 训练脚本内部测试：模型权重加载成功，文件有效！")
except Exception as e:
    print(f" 训练脚本内部测试：模型权重加载失败！文件可能损坏或与模型定义不匹配: {e}")

✔ 训练脚本内部测试：模型权重加载成功，文件有效！


In [31]:
#!/usr/bin/env python3
# generate_single_sts.py

import os
import glob
import numpy as np
import pandas as pd
import torch
import torch.nn as nn
import torch.optim as optim

INPUT_DIR = r'E:/学习工作/PD/pks/SitToStand/Data/STS_2D_skeletons_coarsened'

INPUT_FILE_PATTERN = r'E:/学习工作/PD/pks/SitToStand/Data/STS_2D_skeletons_coarsened/**/*.csv'



SEQ_LEN   = 300
NUM_KPT   = 25
INPUT_DIM = NUM_KPT * 2

DEVICE = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {DEVICE}")

# ------------ Positional Encoding ------------
class PositionalEncoding(nn.Module):
    def __init__(self, d_model, max_len=1000):
        super().__init__()
        pe = torch.zeros(max_len, d_model)
        pos = torch.arange(0, max_len).unsqueeze(1).float()
        div = torch.exp(torch.arange(0, d_model, 2).float() * -(np.log(10000.0) / d_model))
        pe[:, 0::2] = torch.sin(pos * div)
        pe[:, 1::2] = torch.cos(pos * div)
        self.register_buffer('pe', pe.unsqueeze(0))

    def forward(self, x):
        # x: (B, L, D)
        return x + self.pe[:, :x.size(1)]


class TransformerVAE(nn.Module):
    def __init__(self, input_dim, seq_len, d_model=128, nhead=4, num_layers=2, latent_dim=64):
        super().__init__()
        self.seq_len = seq_len
        self.input_linear = nn.Linear(input_dim, d_model)
        self.pos_enc = PositionalEncoding(d_model, max_len=seq_len)
        
        encoder_layer = nn.TransformerEncoderLayer(d_model, nhead, batch_first=True)
        self.encoder = nn.TransformerEncoder(encoder_layer, num_layers)
        
        self.fc_mu = nn.Linear(d_model * seq_len, latent_dim)
        self.fc_logvar = nn.Linear(d_model * seq_len, latent_dim)
        
        self.fc_latent_to_d_model = nn.Linear(latent_dim, d_model)
        
        decoder_layer = nn.TransformerDecoderLayer(d_model, nhead, batch_first=True)
        self.decoder = nn.TransformerDecoder(decoder_layer, num_layers)
        self.output_linear = nn.Linear(d_model, input_dim)

    def encode(self, x):
        # x: (B, L, D_input)
        h = self.input_linear(x)
        h_with_pos = self.pos_enc(h)
        encoder_output = self.encoder(h_with_pos)
        flat_encoder_output = encoder_output.contiguous().view(encoder_output.size(0), -1)
        mu = self.fc_mu(flat_encoder_output)
        logvar = self.fc_logvar(flat_encoder_output)
        return mu, logvar, encoder_output

    def reparameterize(self, mu, logvar):
        std = torch.exp(0.5 * logvar)
        eps = torch.randn_like(std)
        return mu + eps * std

    def decode(self, z, encoder_memory=None):
        # z: (B, latent_dim)
        if encoder_memory is not None:
            initial_tgt_input = self.fc_latent_to_d_model(z).unsqueeze(1).repeat(1, self.seq_len, 1)
            tgt_with_pos = self.pos_enc(initial_tgt_input)
            out = self.decoder(tgt_with_pos, encoder_memory)
        else:
            x = z.view(z.size(0), self.seq_len, -1) if hasattr(self, 'fc_latent') else self.fc_latent_to_d_model(z).unsqueeze(1).repeat(1, self.seq_len, 1)
            tgt = self.pos_enc(x)
            memory = torch.zeros(self.seq_len, z.size(0), tgt.size(2), device=z.device)
            out = self.decoder(tgt.transpose(0,1), memory).transpose(0,1)
        return self.output_linear(out)


def process_single_file(input_file_path, output_dir, model_path=None, scaler_path=None):
    """处理单个CSV文件并生成合成数据"""
    try:
        print(f"正在处理文件: {input_file_path}")
        
        df = pd.read_csv(input_file_path, header=2, engine='python', on_bad_lines='skip')
        print(f"CSV 文件读取完成，形状为: {df.shape}")

        if df.shape[1] < 2 + INPUT_DIM:
            print(f"警告: 文件 {input_file_path} 的列数不足，跳过处理")
            return False

        meta = df.iloc[:, :2].reset_index(drop=True)
        data_np = df.iloc[:, 2:2+INPUT_DIM].to_numpy(dtype=np.float32)
        orig_len = data_np.shape[0]
        print(f"原始数据长度: {orig_len}")

        if np.isnan(data_np).any():
            print("检测到NaN值，进行填充...")
            data_np = np.nan_to_num(data_np, nan=0.0)

        scaler = None
        if scaler_path and os.path.exists(scaler_path):
            print(f"加载训练好的scaler: {scaler_path}")
            import pickle
            with open(scaler_path, 'rb') as f:
                scaler = pickle.load(f)
            data_np_normalized = scaler.transform(data_np)
        else:
            print("未找到scaler文件，使用原始数据（可能导致结果不准确）")
            data_np_normalized = data_np

        if orig_len < SEQ_LEN:
            pad = np.zeros((SEQ_LEN - orig_len, INPUT_DIM), dtype=np.float32)
            data_for_model = np.vstack([data_np_normalized, pad])
        else:
            data_for_model = data_np_normalized[:SEQ_LEN]

        model = TransformerVAE(INPUT_DIM, SEQ_LEN).to(DEVICE)
        
        if model_path and os.path.exists(model_path):
            print(f"加载训练好的模型: {model_path}")
            model.load_state_dict(torch.load(model_path, map_location=DEVICE))
            model.eval()
        else:
            print("警告：未找到训练好的模型，使用随机初始化的模型（结果将是随机的）")
            model.eval()

        with torch.no_grad():
            x = torch.from_numpy(data_for_model[None]).to(DEVICE)
            if hasattr(model, 'encode') and len(model.encode(x)) == 3:
                mu, logvar, encoder_output = model.encode(x)
                z = model.reparameterize(mu, logvar)
                synth = model.decode(z, encoder_output)[0].cpu().numpy()
            else:
                mu, logvar = model.encode(x)
                z = model.reparameterize(mu, logvar)
                synth = model.decode(z)[0].cpu().numpy()

        if scaler is not None:
            print("对生成的数据进行反归一化...")
            synth_for_inverse = synth[:orig_len] if orig_len < SEQ_LEN else synth
            synth_denormalized = scaler.inverse_transform(synth_for_inverse)
            
            if orig_len < SEQ_LEN:
                pad_back = np.zeros((SEQ_LEN - orig_len, INPUT_DIM), dtype=np.float32)
                synth = np.vstack([synth_denormalized, pad_back])
            else:
                synth = synth_denormalized
        else:
            print("跳过反归一化步骤")

        if orig_len > SEQ_LEN:
            pad_back = np.zeros((orig_len - SEQ_LEN, INPUT_DIM), dtype=np.float32)
            full_synth = np.vstack([synth, pad_back])
        else:
            full_synth = synth[:orig_len]

        print(f"Meta shape: {meta.shape}, Full synth shape: {full_synth.shape}")
        
        if meta.shape[0] != full_synth.shape[0]:
            if meta.shape[0] > full_synth.shape[0]:
                pad_back = np.zeros((meta.shape[0] - full_synth.shape[0], INPUT_DIM), dtype=np.float32)
                full_synth = np.vstack([full_synth, pad_back])
            else:
                full_synth = full_synth[:meta.shape[0]]

        out_arr = np.hstack([meta.values, full_synth])
        out_cols = list(meta.columns) + list(df.columns[2:2+INPUT_DIM])
        out_df = pd.DataFrame(out_arr, columns=out_cols)

        os.makedirs(output_dir, exist_ok=True)
        
        input_filename = os.path.basename(input_file_path)
        base_name, ext = os.path.splitext(input_filename)
        output_filename = f"{base_name}_synthetic{ext}"
        output_path = os.path.join(output_dir, output_filename)
        
        out_df.to_csv(output_path, header=False, index=False, float_format='%.6f')
        print(f"✔ 合成结果已保存：{output_path} （shape={out_df.shape}）")
        return True
        
    except Exception as e:
        print(f" 处理文件 {input_file_path} 时出错: {str(e)}")
        return False


def main():
    print("开始批量处理CSV文件...")
    
    all_input_files = glob.glob(INPUT_FILE_PATTERN, recursive=True)
    
    if not all_input_files:
        print(f"没有找到匹配模式 '{INPUT_FILE_PATTERN}' 的文件。请检查路径和模式是否正确。")
        
        direct_pattern = os.path.join(INPUT_DIR, "*.csv")
        all_input_files = glob.glob(direct_pattern)
        
        if not all_input_files:
            print(f"也没有在 '{INPUT_DIR}' 中找到CSV文件。")
            print("请检查以下几点：")
            print("1. 路径是否正确")
            print("2. 文件夹中是否包含CSV文件")
            print("3. 是否有足够的文件访问权限")
            return
        else:
            print(f"在主目录中找到 {len(all_input_files)} 个CSV文件")
    else:
        print(f"找到 {len(all_input_files)} 个匹配文件")
    
    success_count = 0
    for i, file_path in enumerate(all_input_files):
        print(f"\n正在处理第 {i+1}/{len(all_input_files)} 个文件...")
        if process_single_file(file_path, OUTPUT_DIR, MODEL_PATH, SCALER_PATH):
            success_count += 1
    
    print(f"\n处理完成！成功处理 {success_count}/{len(all_input_files)} 个文件")

if __name__ == "__main__":
    main()

Using device: cuda
开始批量处理CSV文件...
找到 404 个匹配文件

正在处理第 1/404 个文件...
正在处理文件: E:/学习工作/PD/pks/SitToStand/Data/STS_2D_skeletons_coarsened\Pt204_C_n_301.csv
CSV 文件读取完成，形状为: (508, 52)
原始数据长度: 508
加载训练好的scaler: scaler_segmented.pkl
加载训练好的模型: transformer_vae_weights.pth
对生成的数据进行反归一化...
Meta shape: (508, 2), Full synth shape: (508, 50)
✔ 合成结果已保存：E:/学习工作/PD/pks/SitToStand/Data/Transformer_generate\Pt204_C_n_301_synthetic.csv （shape=(508, 52)）

正在处理第 2/404 个文件...
正在处理文件: E:/学习工作/PD/pks/SitToStand/Data/STS_2D_skeletons_coarsened\Pt204_C_n_301_synthetic.csv


C:\Users\taro\AppData\Local\Temp\ipykernel_2640\3603144953.py:153: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  model.load_state_dict(torch.load(model_path, map_location=DE

CSV 文件读取完成，形状为: (505, 52)
原始数据长度: 505
加载训练好的scaler: scaler_segmented.pkl
加载训练好的模型: transformer_vae_weights.pth
对生成的数据进行反归一化...
Meta shape: (505, 2), Full synth shape: (505, 50)
✔ 合成结果已保存：E:/学习工作/PD/pks/SitToStand/Data/Transformer_generate\Pt204_C_n_301_synthetic_synthetic.csv （shape=(505, 52)）

正在处理第 3/404 个文件...
正在处理文件: E:/学习工作/PD/pks/SitToStand/Data/STS_2D_skeletons_coarsened\Pt204_C_n_305.csv
CSV 文件读取完成，形状为: (508, 52)
原始数据长度: 508
加载训练好的scaler: scaler_segmented.pkl


C:\Users\taro\AppData\Local\Temp\ipykernel_2640\3603144953.py:153: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  model.load_state_dict(torch.load(model_path, map_location=DE

加载训练好的模型: transformer_vae_weights.pth
对生成的数据进行反归一化...
Meta shape: (508, 2), Full synth shape: (508, 50)
✔ 合成结果已保存：E:/学习工作/PD/pks/SitToStand/Data/Transformer_generate\Pt204_C_n_305_synthetic.csv （shape=(508, 52)）

正在处理第 4/404 个文件...
正在处理文件: E:/学习工作/PD/pks/SitToStand/Data/STS_2D_skeletons_coarsened\Pt204_C_n_306.csv
CSV 文件读取完成，形状为: (508, 52)
原始数据长度: 508
加载训练好的scaler: scaler_segmented.pkl
加载训练好的模型: transformer_vae_weights.pth
对生成的数据进行反归一化...
Meta shape: (508, 2), Full synth shape: (508, 50)
✔ 合成结果已保存：E:/学习工作/PD/pks/SitToStand/Data/Transformer_generate\Pt204_C_n_306_synthetic.csv （shape=(508, 52)）

正在处理第 5/404 个文件...
正在处理文件: E:/学习工作/PD/pks/SitToStand/Data/STS_2D_skeletons_coarsened\Pt204_C_n_307.csv
CSV 文件读取完成，形状为: (101, 52)
原始数据长度: 101
加载训练好的scaler: scaler_segmented.pkl


...

C:\Users\taro\AppData\Local\Temp\ipykernel_2640\3603144953.py:153: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  model.load_state_dict(torch.load(model_path, map_location=DE

✔ 合成结果已保存：E:/学习工作/PD/pks/SitToStand/Data/Transformer_generate\Pt971_C_n_813_synthetic.csv （shape=(508, 52)）

正在处理第 401/404 个文件...
正在处理文件: E:/学习工作/PD/pks/SitToStand/Data/STS_2D_skeletons_coarsened\Pt971_C_n_814.csv
CSV 文件读取完成，形状为: (508, 52)
原始数据长度: 508
加载训练好的scaler: scaler_segmented.pkl
加载训练好的模型: transformer_vae_weights.pth
对生成的数据进行反归一化...
Meta shape: (508, 2), Full synth shape: (508, 50)
✔ 合成结果已保存：E:/学习工作/PD/pks/SitToStand/Data/Transformer_generate\Pt971_C_n_814_synthetic.csv （shape=(508, 52)）

正在处理第 402/404 个文件...
正在处理文件: E:/学习工作/PD/pks/SitToStand/Data/STS_2D_skeletons_coarsened\Pt971_C_n_815.csv
CSV 文件读取完成，形状为: (508, 52)
原始数据长度: 508
加载训练好的scaler: scaler_segmented.pkl
加载训练好的模型: transformer_vae_weights.pth
对生成的数据进行反归一化...
Meta shape: (508, 2), Full synth shape: (508, 50)
✔ 合成结果已保存：E:/学习工作/PD/pks/SitToStand/Data/Transformer_generate\Pt971_C_n_815_synthetic.csv （shape=(508, 52)）

正在处理第 403/404 个文件...
正在处理文件: E:/学习工作/PD/pks/SitToStand/Data/STS_2D_skeletons_coarsened\Pt971_C_n_816.csv
CSV 

C:\Users\taro\AppData\Local\Temp\ipykernel_2640\3603144953.py:153: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  model.load_state_dict(torch.load(model_path, map_location=DE

对生成的数据进行反归一化...
Meta shape: (508, 2), Full synth shape: (508, 50)
✔ 合成结果已保存：E:/学习工作/PD/pks/SitToStand/Data/Transformer_generate\Pt971_C_n_816_synthetic.csv （shape=(508, 52)）

正在处理第 404/404 个文件...
正在处理文件: E:/学习工作/PD/pks/SitToStand/Data/STS_2D_skeletons_coarsened\Pt971_C_n_817.csv
CSV 文件读取完成，形状为: (508, 52)
原始数据长度: 508
加载训练好的scaler: scaler_segmented.pkl
加载训练好的模型: transformer_vae_weights.pth
对生成的数据进行反归一化...
Meta shape: (508, 2), Full synth shape: (508, 50)
✔ 合成结果已保存：E:/学习工作/PD/pks/SitToStand/Data/Transformer_generate\Pt971_C_n_817_synthetic.csv （shape=(508, 52)）

处理完成！成功处理 404/404 个文件


C:\Users\taro\AppData\Local\Temp\ipykernel_2640\3603144953.py:153: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  model.load_state_dict(torch.load(model_path, map_location=DE

In [29]:
#!/usr/bin/env python3
# generate_single_sts.py

import os
import glob
import numpy as np
import pandas as pd
import torch
import torch.nn as nn
import torch.optim as optim

INPUT_DIR = r'E:/学习工作/PD/pks/SitToStand/Data/STS_2D_skeletons_coarsened'

INPUT_FILE_PATTERN = r'E:/学习工作/PD/pks/SitToStand/Data/STS_2D_skeletons_coarsened/**/*.csv'


SEQ_LEN   = 50
NUM_KPT   = 25
INPUT_DIM = NUM_KPT * 2

DEVICE = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {DEVICE}")

# ------------ Positional Encoding ------------
class PositionalEncoding(nn.Module):
    def __init__(self, d_model, max_len=1000):
        super().__init__()
        pe = torch.zeros(max_len, d_model)
        pos = torch.arange(0, max_len).unsqueeze(1).float()
        div = torch.exp(torch.arange(0, d_model, 2).float() * -(np.log(10000.0) / d_model))
        pe[:, 0::2] = torch.sin(pos * div)
        pe[:, 1::2] = torch.cos(pos * div)
        self.register_buffer('pe', pe.unsqueeze(0))

    def forward(self, x):
        # x: (B, L, D)
        return x + self.pe[:, :x.size(1)]


# ------------ Transformer VAE ------------
class TransformerVAE(nn.Module):
    def __init__(self, input_dim, seq_len, d_model=128, nhead=4, num_layers=2, latent_dim=64):
        super().__init__()
        self.seq_len = seq_len
        self.input_linear = nn.Linear(input_dim, d_model)
        self.pos_enc      = PositionalEncoding(d_model, max_len=seq_len)
        encoder_layer     = nn.TransformerEncoderLayer(d_model, nhead)
        self.encoder      = nn.TransformerEncoder(encoder_layer, num_layers)
        self.fc_mu        = nn.Linear(d_model*seq_len, latent_dim)
        self.fc_logvar    = nn.Linear(d_model*seq_len, latent_dim)
        self.fc_latent    = nn.Linear(latent_dim, d_model*seq_len)
        decoder_layer     = nn.TransformerDecoderLayer(d_model, nhead)
        self.decoder      = nn.TransformerDecoder(decoder_layer, num_layers)
        self.output_linear= nn.Linear(d_model, input_dim)

    def encode(self, x):
        # x: (B, L, D)
        h = self.input_linear(x)
        h = self.pos_enc(h)
        out = self.encoder(h.transpose(0,1)).transpose(0,1)
        flat = out.contiguous().view(out.size(0), -1)
        return self.fc_mu(flat), self.fc_logvar(flat)

    def reparameterize(self, mu, logvar):
        std = torch.exp(0.5 * logvar)
        eps = torch.randn_like(std)
        return mu + eps * std

    def decode(self, z):
        # z: (B, latent_dim)
        x = self.fc_latent(z).view(z.size(0), self.seq_len, -1)
        tgt = self.pos_enc(x)
        memory = torch.zeros(self.seq_len, z.size(0), tgt.size(2), device=z.device)
        out = self.decoder(tgt.transpose(0,1), memory).transpose(0,1)
        return self.output_linear(out)


def process_single_file(input_file_path, output_dir):
    """处理单个CSV文件并生成合成数据"""
    try:
        print(f"正在处理文件: {input_file_path}")
        
        df = pd.read_csv(input_file_path, header=2, engine='python', on_bad_lines='skip')
        print(f"CSV 文件读取完成，形状为: {df.shape}")

        if df.shape[1] < 2 + INPUT_DIM:
            print(f"警告: 文件 {input_file_path} 的列数不足，跳过处理")
            return False

        meta = df.iloc[:, :2].reset_index(drop=True)
        data_np = df.iloc[:, 2:2+INPUT_DIM].to_numpy(dtype=np.float32)
        orig_len = data_np.shape[0]
        print(f"原始数据长度: {orig_len}")

        if np.isnan(data_np).any():
            print("检测到NaN值，进行填充...")
            data_np = np.nan_to_num(data_np, nan=0.0)

        if orig_len < SEQ_LEN:
            pad = np.zeros((SEQ_LEN - orig_len, INPUT_DIM), dtype=np.float32)
            data_for_model = np.vstack([data_np, pad])
        else:
            data_for_model = data_np[:SEQ_LEN]

        print("初始化模型...")
        model = TransformerVAE(INPUT_DIM, SEQ_LEN).to(DEVICE)
        model.eval()

        with torch.no_grad():
            x = torch.from_numpy(data_for_model[None]).to(DEVICE)
            mu, logvar = model.encode(x)
            z = model.reparameterize(mu, logvar)
            synth = model.decode(z)[0].cpu().numpy()

        if orig_len > SEQ_LEN:
            pad_back = np.zeros((orig_len - SEQ_LEN, INPUT_DIM), dtype=np.float32)
            full_synth = np.vstack([synth, pad_back])
        else:
            full_synth = synth[:orig_len]

        print(f"Meta shape: {meta.shape}, Full synth shape: {full_synth.shape}")
        
        if meta.shape[0] != full_synth.shape[0]:
            if meta.shape[0] > full_synth.shape[0]:
                pad_back = np.zeros((meta.shape[0] - full_synth.shape[0], INPUT_DIM), dtype=np.float32)
                full_synth = np.vstack([full_synth, pad_back])
            else:
                full_synth = full_synth[:meta.shape[0]]

        out_arr = np.hstack([meta.values, full_synth])
        out_cols = list(meta.columns) + list(df.columns[2:2+INPUT_DIM])
        out_df = pd.DataFrame(out_arr, columns=out_cols)

        os.makedirs(output_dir, exist_ok=True)
        
        input_filename = os.path.basename(input_file_path)
        base_name, ext = os.path.splitext(input_filename)
        output_filename = f"{base_name}_synthetic{ext}"
        output_path = os.path.join(output_dir, output_filename)
        
        out_df.to_csv(output_path, header=False, index=False, float_format='%.6f')
        print(f"✔ 合成结果已保存：{output_path} （shape={out_df.shape}）")
        return True
        
    except Exception as e:
        print(f" 处理文件 {input_file_path} 时出错: {str(e)}")
        return False


def main():
    print("开始批量处理CSV文件...")
    
    all_input_files = glob.glob(INPUT_FILE_PATTERN, recursive=True)
    
    if not all_input_files:
        print(f"没有找到匹配模式 '{INPUT_FILE_PATTERN}' 的文件。请检查路径和模式是否正确。")
        
        direct_pattern = os.path.join(INPUT_DIR, "*.csv")
        all_input_files = glob.glob(direct_pattern)
        
        if not all_input_files:
            print(f"也没有在 '{INPUT_DIR}' 中找到CSV文件。")
            print("请检查以下几点：")
            print("1. 路径是否正确")
            print("2. 文件夹中是否包含CSV文件")
            print("3. 是否有足够的文件访问权限")
            return
        else:
            print(f"在主目录中找到 {len(all_input_files)} 个CSV文件")
    else:
        print(f"找到 {len(all_input_files)} 个匹配文件")
    
    success_count = 0
    for i, file_path in enumerate(all_input_files):
        print(f"\n正在处理第 {i+1}/{len(all_input_files)} 个文件...")
        if process_single_file(file_path, OUTPUT_DIR):
            success_count += 1
    
    print(f"\n处理完成！成功处理 {success_count}/{len(all_input_files)} 个文件")

if __name__ == "__main__":
    main()

Using device: cuda
开始批量处理CSV文件...
找到 404 个匹配文件

正在处理第 1/404 个文件...
正在处理文件: E:/学习工作/PD/pks/SitToStand/Data/STS_2D_skeletons_coarsened\Pt204_C_n_301.csv
CSV 文件读取完成，形状为: (508, 52)
原始数据长度: 508
初始化模型...
Meta shape: (508, 2), Full synth shape: (508, 50)
✔ 合成结果已保存：E:/学习工作/PD/pks/SitToStand/Data/Transformer_generate\Pt204_C_n_301_synthetic.csv （shape=(508, 52)）

正在处理第 2/404 个文件...
正在处理文件: E:/学习工作/PD/pks/SitToStand/Data/STS_2D_skeletons_coarsened\Pt204_C_n_301_synthetic.csv
CSV 文件读取完成，形状为: (505, 52)
原始数据长度: 505
初始化模型...
Meta shape: (505, 2), Full synth shape: (505, 50)
✔ 合成结果已保存：E:/学习工作/PD/pks/SitToStand/Data/Transformer_generate\Pt204_C_n_301_synthetic_synthetic.csv （shape=(505, 52)）

正在处理第 3/404 个文件...
正在处理文件: E:/学习工作/PD/pks/SitToStand/Data/STS_2D_skeletons_coarsened\Pt204_C_n_305.csv
CSV 文件读取完成，形状为: (508, 52)
原始数据长度: 508
初始化模型...


e:\python3.10.9\lib\site-packages\torch\nn\modules\transformer.py:379: UserWarning: enable_nested_tensor is True, but self.use_nested_tensor is False because encoder_layer.self_attn.batch_first was not True(use batch_first for better inference performance)
  warnings.warn(
e:\python3.10.9\lib\site-packages\torch\nn\modules\transformer.py:379: UserWarning: enable_nested_tensor is True, but self.use_nested_tensor is False because encoder_layer.self_attn.batch_first was not True(use batch_first for better inference performance)
  warnings.warn(
e:\python3.10.9\lib\site-packages\torch\nn\modules\transformer.py:379: UserWarning: enable_nested_tensor is True, but self.use_nested_tensor is False because encoder_layer.self_attn.batch_first was not True(use batch_first for better inference performance)
  warnings.warn(


Meta shape: (508, 2), Full synth shape: (508, 50)
✔ 合成结果已保存：E:/学习工作/PD/pks/SitToStand/Data/Transformer_generate\Pt204_C_n_305_synthetic.csv （shape=(508, 52)）

正在处理第 4/404 个文件...
正在处理文件: E:/学习工作/PD/pks/SitToStand/Data/STS_2D_skeletons_coarsened\Pt204_C_n_306.csv
CSV 文件读取完成，形状为: (508, 52)
原始数据长度: 508
初始化模型...
Meta shape: (508, 2), Full synth shape: (508, 50)
✔ 合成结果已保存：E:/学习工作/PD/pks/SitToStand/Data/Transformer_generate\Pt204_C_n_306_synthetic.csv （shape=(508, 52)）

正在处理第 5/404 个文件...
正在处理文件: E:/学习工作/PD/pks/SitToStand/Data/STS_2D_skeletons_coarsened\Pt204_C_n_307.csv
CSV 文件读取完成，形状为: (101, 52)
原始数据长度: 101
初始化模型...
Meta shape: (101, 2), Full synth shape: (101, 50)
✔ 合成结果已保存：E:/学习工作/PD/pks/SitToStand/Data/Transformer_generate\Pt204_C_n_307_synthetic.csv （shape=(101, 52)）

正在处理第 6/404 个文件...
正在处理文件: E:/学习工作/PD/pks/SitToStand/Data/STS_2D_skeletons_coarsened\Pt204_C_n_316.csv
CSV 文件读取完成，形状为: (508, 52)
原始数据长度: 508
初始化模型...


e:\python3.10.9\lib\site-packages\torch\nn\modules\transformer.py:379: UserWarning: enable_nested_tensor is True, but self.use_nested_tensor is False because encoder_layer.self_attn.batch_first was not True(use batch_first for better inference performance)
  warnings.warn(
e:\python3.10.9\lib\site-packages\torch\nn\modules\transformer.py:379: UserWarning: enable_nested_tensor is True, but self.use_nested_tensor is False because encoder_layer.self_attn.batch_first was not True(use batch_first for better inference performance)
  warnings.warn(
e:\python3.10.9\lib\site-packages\torch\nn\modules\transformer.py:379: UserWarning: enable_nested_tensor is True, but self.use_nested_tensor is False because encoder_layer.self_attn.batch_first was not True(use batch_first for better inference performance)
  warnings.warn(


Meta shape: (508, 2), Full synth shape: (508, 50)
✔ 合成结果已保存：E:/学习工作/PD/pks/SitToStand/Data/Transformer_generate\Pt204_C_n_316_synthetic.csv （shape=(508, 52)）

正在处理第 7/404 个文件...
正在处理文件: E:/学习工作/PD/pks/SitToStand/Data/STS_2D_skeletons_coarsened\Pt204_C_n_317.csv
CSV 文件读取完成，形状为: (508, 52)
原始数据长度: 508
初始化模型...
Meta shape: (508, 2), Full synth shape: (508, 50)
✔ 合成结果已保存：E:/学习工作/PD/pks/SitToStand/Data/Transformer_generate\Pt204_C_n_317_synthetic.csv （shape=(508, 52)）

正在处理第 8/404 个文件...
正在处理文件: E:/学习工作/PD/pks/SitToStand/Data/STS_2D_skeletons_coarsened\Pt204_C_n_318.csv
CSV 文件读取完成，形状为: (163, 52)
原始数据长度: 163
初始化模型...
Meta shape: (163, 2), Full synth shape: (163, 50)
✔ 合成结果已保存：E:/学习工作/PD/pks/SitToStand/Data/Transformer_generate\Pt204_C_n_318_synthetic.csv （shape=(163, 52)）

正在处理第 9/404 个文件...
正在处理文件: E:/学习工作/PD/pks/SitToStand/Data/STS_2D_skeletons_coarsened\Pt204_C_n_319.csv
CSV 文件读取完成，形状为: (508, 52)
原始数据长度: 508
初始化模型...
Meta shape: (508, 2), Full synth shape: (508, 50)


...

e:\python3.10.9\lib\site-packages\torch\nn\modules\transformer.py:379: UserWarning: enable_nested_tensor is True, but self.use_nested_tensor is False because encoder_layer.self_attn.batch_first was not True(use batch_first for better inference performance)
  warnings.warn(
e:\python3.10.9\lib\site-packages\torch\nn\modules\transformer.py:379: UserWarning: enable_nested_tensor is True, but self.use_nested_tensor is False because encoder_layer.self_attn.batch_first was not True(use batch_first for better inference performance)
  warnings.warn(
e:\python3.10.9\lib\site-packages\torch\nn\modules\transformer.py:379: UserWarning: enable_nested_tensor is True, but self.use_nested_tensor is False because encoder_layer.self_attn.batch_first was not True(use batch_first for better inference performance)
  warnings.warn(
e:\python3.10.9\lib\site-packages\torch\nn\modules\transformer.py:379: UserWarning: enable_nested_tensor is True, but self.use_nested_tensor is False because encoder_layer.self_a

Meta shape: (508, 2), Full synth shape: (508, 50)
✔ 合成结果已保存：E:/学习工作/PD/pks/SitToStand/Data/Transformer_generate\Pt971_C_n_795_synthetic.csv （shape=(508, 52)）

正在处理第 395/404 个文件...
正在处理文件: E:/学习工作/PD/pks/SitToStand/Data/STS_2D_skeletons_coarsened\Pt971_C_n_796.csv
CSV 文件读取完成，形状为: (237, 52)
原始数据长度: 237
初始化模型...
Meta shape: (237, 2), Full synth shape: (237, 50)
✔ 合成结果已保存：E:/学习工作/PD/pks/SitToStand/Data/Transformer_generate\Pt971_C_n_796_synthetic.csv （shape=(237, 52)）

正在处理第 396/404 个文件...
正在处理文件: E:/学习工作/PD/pks/SitToStand/Data/STS_2D_skeletons_coarsened\Pt971_C_n_797.csv
CSV 文件读取完成，形状为: (188, 52)
原始数据长度: 188
初始化模型...
Meta shape: (188, 2), Full synth shape: (188, 50)
✔ 合成结果已保存：E:/学习工作/PD/pks/SitToStand/Data/Transformer_generate\Pt971_C_n_797_synthetic.csv （shape=(188, 52)）

正在处理第 397/404 个文件...
正在处理文件: E:/学习工作/PD/pks/SitToStand/Data/STS_2D_skeletons_coarsened\Pt971_C_n_802.csv
CSV 文件读取完成，形状为: (508, 52)
原始数据长度: 508
初始化模型...
Meta shape: (508, 2), Full synth shape: (508, 50)
✔ 合成结果已保存：E:/学习工作

e:\python3.10.9\lib\site-packages\torch\nn\modules\transformer.py:379: UserWarning: enable_nested_tensor is True, but self.use_nested_tensor is False because encoder_layer.self_attn.batch_first was not True(use batch_first for better inference performance)
  warnings.warn(
e:\python3.10.9\lib\site-packages\torch\nn\modules\transformer.py:379: UserWarning: enable_nested_tensor is True, but self.use_nested_tensor is False because encoder_layer.self_attn.batch_first was not True(use batch_first for better inference performance)
  warnings.warn(
e:\python3.10.9\lib\site-packages\torch\nn\modules\transformer.py:379: UserWarning: enable_nested_tensor is True, but self.use_nested_tensor is False because encoder_layer.self_attn.batch_first was not True(use batch_first for better inference performance)
  warnings.warn(
e:\python3.10.9\lib\site-packages\torch\nn\modules\transformer.py:379: UserWarning: enable_nested_tensor is True, but self.use_nested_tensor is False because encoder_layer.self_a

Meta shape: (508, 2), Full synth shape: (508, 50)
✔ 合成结果已保存：E:/学习工作/PD/pks/SitToStand/Data/Transformer_generate\Pt971_C_n_815_synthetic.csv （shape=(508, 52)）

正在处理第 403/404 个文件...
正在处理文件: E:/学习工作/PD/pks/SitToStand/Data/STS_2D_skeletons_coarsened\Pt971_C_n_816.csv
CSV 文件读取完成，形状为: (508, 52)
原始数据长度: 508
初始化模型...
Meta shape: (508, 2), Full synth shape: (508, 50)
✔ 合成结果已保存：E:/学习工作/PD/pks/SitToStand/Data/Transformer_generate\Pt971_C_n_816_synthetic.csv （shape=(508, 52)）

正在处理第 404/404 个文件...
正在处理文件: E:/学习工作/PD/pks/SitToStand/Data/STS_2D_skeletons_coarsened\Pt971_C_n_817.csv
CSV 文件读取完成，形状为: (508, 52)
原始数据长度: 508
初始化模型...
Meta shape: (508, 2), Full synth shape: (508, 50)
✔ 合成结果已保存：E:/学习工作/PD/pks/SitToStand/Data/Transformer_generate\Pt971_C_n_817_synthetic.csv （shape=(508, 52)）

处理完成！成功处理 404/404 个文件


e:\python3.10.9\lib\site-packages\torch\nn\modules\transformer.py:379: UserWarning: enable_nested_tensor is True, but self.use_nested_tensor is False because encoder_layer.self_attn.batch_first was not True(use batch_first for better inference performance)
  warnings.warn(
e:\python3.10.9\lib\site-packages\torch\nn\modules\transformer.py:379: UserWarning: enable_nested_tensor is True, but self.use_nested_tensor is False because encoder_layer.self_attn.batch_first was not True(use batch_first for better inference performance)
  warnings.warn(


In [27]:
import os

try:
    # Example to check if the input folder is accessible
    if os.path.exists(INPUT_DIR):
        print(f"Input directory found: {INPUT_DIR}")
    else:
        print(f"Input directory does not exist: {INPUT_DIR}")
except PermissionError as e:
    print(f"Permission error: {e}")
except Exception as e:
    print(f"Unexpected error: {e}")


Input directory found: E:/学习工作/PD/pks/SitToStand/Data/STS_2D_skeletons_coarsened


In [ ]:
#!/usr/bin/env python3
# generate_single_sts.py

import os
import numpy as np
import pandas as pd
import torch
import torch.nn as nn
import torch.optim as optim

INPUT_FILE = r'E:/学习工作/PD/pks/SitToStand/Data/STS_2D_skeletons_coarsened/Pt204_C_n_305.csv'

SEQ_LEN   = 50
NUM_KPT   = 25
INPUT_DIM = NUM_KPT * 2

DEVICE = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using Input File: {INPUT_FILE}")

# ------------ Positional Encoding ------------
class PositionalEncoding(nn.Module):
    def __init__(self, d_model, max_len=1000):
        super().__init__()
        pe = torch.zeros(max_len, d_model)
        pos = torch.arange(0, max_len).unsqueeze(1).float()
        div = torch.exp(torch.arange(0, d_model, 2).float() * -(np.log(10000.0) / d_model))
        pe[:, 0::2] = torch.sin(pos * div)
        pe[:, 1::2] = torch.cos(pos * div)
        self.register_buffer('pe', pe.unsqueeze(0))

    def forward(self, x):
        # x: (B, L, D)
        return x + self.pe[:, :x.size(1)]


# ------------ Transformer VAE ------------
class TransformerVAE(nn.Module):
    def __init__(self, input_dim, seq_len, d_model=128, nhead=4, num_layers=2, latent_dim=64):
        super().__init__()
        self.seq_len = seq_len
        self.input_linear = nn.Linear(input_dim, d_model)
        self.pos_enc      = PositionalEncoding(d_model, max_len=seq_len)
        encoder_layer     = nn.TransformerEncoderLayer(d_model, nhead)
        self.encoder      = nn.TransformerEncoder(encoder_layer, num_layers)
        self.fc_mu        = nn.Linear(d_model*seq_len, latent_dim)
        self.fc_logvar    = nn.Linear(d_model*seq_len, latent_dim)
        self.fc_latent    = nn.Linear(latent_dim, d_model*seq_len)
        decoder_layer     = nn.TransformerDecoderLayer(d_model, nhead)
        self.decoder      = nn.TransformerDecoder(decoder_layer, num_layers)
        self.output_linear= nn.Linear(d_model, input_dim)

    def encode(self, x):
        # x: (B, L, D)
        h = self.input_linear(x)
        h = self.pos_enc(h)
        out = self.encoder(h.transpose(0,1)).transpose(0,1)
        flat = out.contiguous().view(out.size(0), -1)
        return self.fc_mu(flat), self.fc_logvar(flat)

    def reparameterize(self, mu, logvar):
        std = torch.exp(0.5 * logvar)
        eps = torch.randn_like(std)
        return mu + eps * std

    def decode(self, z):
        # z: (B, latent_dim)
        x = self.fc_latent(z).view(z.size(0), self.seq_len, -1)
        tgt = self.pos_enc(x)
        memory = torch.zeros(self.seq_len, z.size(0), tgt.size(2), device=z.device)
        out = self.decoder(tgt.transpose(0,1), memory).transpose(0,1)
        return self.output_linear(out)


def main():

    print("开始读取 CSV 文件...")
    df = pd.read_csv(INPUT_FILE, header=2, engine='python', on_bad_lines='skip')
    print(f"CSV 文件读取完成，形状为: {df.shape}")

    meta = df.iloc[:, :2].reset_index(drop=True)
    data_np = df.iloc[:, 2:2+INPUT_DIM].to_numpy(dtype=np.float32)
    orig_len = data_np.shape[0]
    print(f"原始数据长度: {orig_len}")

    if orig_len < SEQ_LEN:
        pad = np.zeros((SEQ_LEN - orig_len, INPUT_DIM), dtype=np.float32)
        data_for_model = np.vstack([data_np, pad])
    else:
        data_for_model = data_np[:SEQ_LEN]

    print("初始化模型...")
    model = TransformerVAE(INPUT_DIM, SEQ_LEN).to(DEVICE)
    model.to(DEVICE).eval()

    with torch.no_grad():
        x = torch.from_numpy(data_for_model[None]).to(DEVICE)
        mu, logvar = model.encode(x)
        z = model.reparameterize(mu, logvar)
        synth = model.decode(z)[0].cpu().numpy()

    if orig_len > SEQ_LEN:
        pad_back = np.zeros((orig_len - SEQ_LEN, INPUT_DIM), dtype=np.float32)
        full_synth = np.vstack([synth, pad_back])
    else:
        full_synth = synth[:orig_len]

    print(f"Meta shape: {meta.shape}, Full synth shape: {full_synth.shape}")
    if meta.shape[0] != full_synth.shape[0]:
        if meta.shape[0] > full_synth.shape[0]:
            pad_back = np.zeros((meta.shape[0] - full_synth.shape[0], INPUT_DIM), dtype=np.float32)
            full_synth = np.vstack([full_synth, pad_back])
        else:
            full_synth = full_synth[:meta.shape[0]]

    out_arr = np.hstack([meta.values, full_synth])
    out_cols = list(meta.columns) + list(df.columns[2:2+INPUT_DIM])
    out_df = pd.DataFrame(out_arr, columns=out_cols)

    if OUTPUT_DIR is None or OUTPUT_DIR == '':
        base, _ = os.path.splitext(INPUT_FILE)
        OUTPUT_DIR = f"{base}_synthetic"

    os.makedirs(OUTPUT_DIR, exist_ok=True)
    out_df.to_csv(os.path.join(OUTPUT_DIR, "synthetic.csv"), header=False, index=False, float_format='%.6f')
    print(f"✔ 合成结果已保存：{OUTPUT_DIR} （shape={out_df.shape}）")
    
if __name__ == "__main__":
    main()